In [4]:
import ccxt
import pandas as pd
import pandas_ta as ta
import numpy as np
from datetime import datetime
import time
import sys
from Analysis import Analysis 
from getSignals import getSignals
from Plot import Plot
import os

In [18]:
data_file = 'BTC_15'
data_file_ent = []
for file in os.listdir(data_file):
    if file.endswith('.csv'):
        data_file_ent.append(file)

In [25]:
for file in data_file_ent:
    if str(1) in file:
        df = pd.read_csv('BTC'+'1'+'.csv')
df

FileNotFoundError: [Errno 2] No such file or directory: 'BTC1.csv'